This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [3]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
# %pip install selenium chromedriver-binary

                                              0.0/139.3 kB ? eta -:--:--
     -----------------                       61.4/139.3 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 139.3/139.3 kB 1.7 MB/s eta 0:00:00
                                              0.0/12.0 MB ? eta -:--:--
                                              0.2/12.0 MB 4.6 MB/s eta 0:00:03
     -                                        0.5/12.0 MB 5.2 MB/s eta 0:00:03
     --                                       0.7/12.0 MB 5.6 MB/s eta 0:00:03
     --                                       0.8/12.0 MB 4.8 MB/s eta 0:00:03
     -----                                    1.6/12.0 MB 7.2 MB/s eta 0:00:02
     -----                                    1.6/12.0 MB 5.7 MB/s eta 0:00:02
     ------                                   2.1/12.0 MB 6.6 MB/s eta 0:00:02
     -------                                  2.4/12.0 MB 6.6 MB/s eta 0:00:02
     -------                                  2.4/12.0 MB 6.6 M


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import boto3

In [2]:
%pip install selenium==4.7.2
%pip install chromedriver-autoinstaller==0.6.4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# The file download processing section should be placed here

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions()
options.add_argument("--lang=vi")
current_directory = os.getcwd()
options.add_experimental_option("prefs", {
  "download.default_directory": current_directory,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing")
    element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "KL4NAf"))
    )
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

    element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[aria-label="Tải xuống"]'))
    )
    element.click()
    print("Download successfully")
except Exception as err:
    print(f"Error: {err}")


Download successfully


In [4]:
import zipfile, os

def unzip_file(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

zip_file_path = 'data.zip' 
extracted_folder = 'data-raw' 
unzip_file(zip_file_path, extracted_folder)

In [5]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'ViObtRdRJI9cs52kg2GK')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'IrAUC7JpogDEMdvnE98BOjqng9LYCPU9cG2mLNU1')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [6]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


data-raw
data-raw\__MACOSX


In [19]:
minio_bucket = 'data'
files = [pos_json for pos_json in os.listdir(extracted_folder) if pos_json.endswith('.json')]
print(files)
for file in files:
    file_path = os.path.join(extracted_folder, file)
    minio_object_name = f"{extracted_folder}/{file}"
    try:
        upload_file_to_minio(file_path, minio_bucket, minio_object_name)
        print(f"Uploaded {file_path} to Minio")
    except Exception as err:
        print(err)

['data.json', 'data2.json', 'data3.json']
file_path:  data-raw\data.json
minio_object_name:  data-raw/data.json
Uploaded data-raw\data.json to Minio
file_path:  data-raw\data2.json
minio_object_name:  data-raw/data2.json
Uploaded data-raw\data2.json to Minio
file_path:  data-raw\data3.json
minio_object_name:  data-raw/data3.json
Uploaded data-raw\data3.json to Minio


In [8]:
# Upload file to minio
# upload_file_to_minio('data-raw/data.json', 'data', 'data-raw/data.json')
# upload_file_to_minio('data-raw/data2.json', 'data', 'data-raw/data2.json')
# upload_file_to_minio('data-raw/data3.json', 'data', 'data-raw/data3.json')